In [27]:
#IMPORTS
from copy import deepcopy
from typing import Tuple, List
import numpy as np
import random as rd
import math
from IPython.display import clear_output

In [28]:
class Tablero:
    def __init__(self, matrix):
        self.setMatrix(matrix)
        self.punt = 0
    
    def __eq__(self, other: 'Tablero') -> bool:
        return (self.getMatrix()==other.getMatrix()).all()

    def setMatrix(self, matrix) -> None:
        self.matrix = deepcopy(np.array(matrix))
    
    def getMatrix(self) -> List[List]:
        return deepcopy(self.matrix)
    
    def placeTile(self, row: int, col: int, tile: int) -> None:
        self.matrix[row][col] = tile

    def getNeighbors(self, i, j) -> List:
        vecinos = []
        for fil in range(-1, 2, +1):
            for col in range(-1, 2, +1):
                if i + fil >= 0 and i + fil < 4 and j + col >= 0 and j + col < 4:
                    vecinos.append(self.matrix[i + fil][j + col])
        return vecinos

    def similitud(self) -> float:
        sim = 0
        for i in range(0,4):
            for j in range(0,4):
                if self.matrix[i][j]:
                    sum = 0
                    vecinos = self.getNeighbors(i, j)
                    l = 0
                    for v in vecinos:
                        if v != 0:
                            sum += abs(self.matrix[i][j] - v)
                            l += 1
                    if l != 0:
                        sim += sum / l
        return sim

    def monotonia(self) -> int:
        matrizMonotonia = np.array([
            [7,6,5,4],
            [6,4,3,3],
            [5,3,3,2],
            [4,3,2,1]
        ])

        return np.sum(np.multiply(self.matrix, matrizMonotonia))

    def snake(self) -> int:
        matrizSerpiente = np.array([
            [4**15,4**8,4**7,4**0],
            [4**14,4**9,4**6,4**1],
            [4**13,4**10,4**5,4**2],
            [4**12,4**11,4**4,4**3]
        ])

        return np.sum(np.multiply(self.matrix, matrizSerpiente))

    def maxInCorner(self) -> int:
        if(self.matrix[0][0] != self.maxNumber()):
            return 0
        return 1 

    def celdasVacias(self):
        return 16 - np.count_nonzero(self.matrix)

    def maxNumber(self) -> int:
        return self.matrix.max()

    def max2048(self) -> int:
        if self.maxNumber() == 2048:
            return 10000
        return 1

    def utility(self) -> int:
        score = 0
        if not self.punt:
            score = 1
        else:
            score = self.punt
        #return np.sum(self.matrix) / (16 - self.celdasVacias())
        #return self.punt + self.monotonia() - self.similitud() + (math.log(score) * self.celdasVacias())
        #return self.monotonia() - self.similitud() + (math.log(score) * self.celdasVacias())
        #return (score - self.similitud()) + (self.celdasVacias() * math.log2(self.maxNumber())) + self.monotonia()
        #return score + (self.celdasVacias() * math.log2(self.maxNumber())) + self.monotonia()
        return self.snake() * self.maxInCorner() * self.max2048()

    def canMoveUp(self) -> bool:
        for col in range(4):
            for fil in range(3, 0, -1):
                if(self.matrix[fil][col] and
                   (self.matrix[fil-1][col] == 0 or
                   self.matrix[fil-1][col] == self.matrix[fil][col])):
                    return True
        return False

    def canMoveDown(self) -> bool:
        for col in range(4):
            for fil in range(0, 3, +1):
                if(self.matrix[fil][col] and
                   (self.matrix[fil+1][col] == 0 or
                   self.matrix[fil+1][col] == self.matrix[fil][col])):
                    return True
        return False

    def canMoveLeft(self) -> bool:
        for fil in range(4):
            for col in range(3, 0, -1):
                if(self.matrix[fil][col] and
                   (self.matrix[fil][col-1] == 0 or
                   self.matrix[fil][col-1] == self.matrix[fil][col])):
                    return True
        return False

    def canMoveRight(self) -> bool:
        for fil in range(4):
            for col in range(0, 3, +1):
                if(self.matrix[fil][col] and
                   (self.matrix[fil][col+1] == 0 or
                   self.matrix[fil][col+1] == self.matrix[fil][col])):
                    return True
        return False
    
    def getAvailableMovesForMax(self) -> List[int]:
        mov = []

        if self.canMoveUp():
            mov.append(0)
        if self.canMoveLeft():
            mov.append(2)
        if self.canMoveDown():
            mov.append(1)
        if self.canMoveRight():
            mov.append(3)
        
        return mov
    
    def getAvailableMovesForMin(self) -> List[Tuple[int]]:
        mov = []

        for i in range(0,4):
            for j in range(0,4):
                if not self.matrix[i][j]:
                    mov.append((i, j, 4))
                    mov.append((i, j, 2))
        return mov

    def getChildren(self, str: str) -> List:
        lista = []
        if str == "max":
            for i in self.getAvailableMovesForMax():
                copia = Tablero(self.getMatrix())
                copia.move(i)
                lista.append(copia)
            return lista
        else:
            for i in self.getAvailableMovesForMin():
                copia = Tablero(self.getMatrix())
                copia.placeTile(i[0], i[1], i[2])
                lista.append(copia)
            return lista


    def isTerminal(self, who: str) -> bool:
        if who == "max":
            if self.canMoveUp():
                return False
            if self.canMoveDown():
                return False
            if self.canMoveLeft():
                return False
            if self.canMoveRight():
                return False
        else:
            if 0 in self.matrix:
                return False
        return True

  
    def isGameOver(self) -> bool:
        return self.isTerminal("max") #and self.isTerminal("max")

    def desplazarCerosDer(self, sublista) -> list:
        #Desplaza todos los 0 (celdas sin pieza) a la derecha. Ej: 0 2 2 0 -> 2 2 0 0
        lista = [n for n in sublista if n != 0]
        for i in range(0, len(sublista) - len(lista)):
            lista.append(0)
        return lista
    
    def desplazarCerosIzq(self, sublista) -> list:
        #Desplaza todos los 0 (celdas sin pieza) a la izquierda. Ej: 0 2 2 0 -> 0 0 2 2
        lista = [n for n in sublista if n != 0]
        for i in range(0, len(sublista) - len(lista)):
            lista.insert(0, 0)
        return lista

    def copiarColumna(self, columna: int, colCp: list) -> None:
        for i in range(0, 4):
            self.matrix[i][columna] = colCp[i]
                
    def up(self) -> None:
        for i in range(0,4):
            columna = self.matrix[:, i]
            actual = 0
            siguiente = 0
            pos = 0
            while pos <= 3:
                columna = self.desplazarCerosDer(columna)
                actual = columna[pos]
                if pos + 1 > 3:
                    break
                siguiente = columna[pos + 1]
                if not siguiente:
                    pos += 1
                    continue
                if actual == siguiente:
                    actual += siguiente
                    self.punt += actual
                    columna[pos + 1] = 0
                    columna[pos] = actual
                pos += 1
            self.copiarColumna(i, columna)  
    
    def down(self) -> None:
        for i in range(0,4):
            columna = self.matrix[:, i]
            actual = 0
            siguiente = 0
            pos = 3
            while pos >= 0:
                columna = self.desplazarCerosIzq(columna)
                actual = columna[pos]
                if pos - 1 < 0:
                    break
                siguiente = columna[pos - 1]
                if not siguiente:
                    pos -= 1
                    continue
                if actual == siguiente:
                    actual += siguiente
                    self.punt += actual
                    columna[pos - 1] = 0
                    columna[pos] = actual
                pos -= 1
            self.copiarColumna(i, columna)  
   
    
    def left(self) -> None:
        for i in range(0,4):
            fila = self.matrix[i]
            actual = 0
            siguiente = 0
            pos = 0
            while pos <= 3:
                fila = self.desplazarCerosDer(fila)
                actual = fila[pos]
                if pos + 1 > 3:
                    break
                siguiente = fila[pos + 1]
                if not siguiente:
                    pos += 1
                    continue
                if actual == siguiente:
                    actual += siguiente
                    self.punt += actual
                    fila[pos + 1] = 0
                    fila[pos] = actual
                pos += 1
            self.matrix[i] = fila
    
    def right(self) -> None:
        for i in range(0,4):
            fila = self.matrix[i]
            actual = 0
            siguiente = 0
            pos = 3
            while pos >= 0:
                fila = self.desplazarCerosIzq(fila)
                actual = fila[pos]
                if pos - 1 < 0:
                    break
                siguiente = fila[pos - 1]
                if not siguiente:
                    pos -= 1
                    continue
                if actual == siguiente:
                    actual += siguiente
                    self.punt += actual
                    fila[pos - 1] = 0
                    fila[pos] = actual
                pos -= 1
            self.matrix[i] = fila
    
    def move(self, mv: int) -> None:
        if not mv:
            self.up()
        elif mv == 1:
            self.down()
        elif mv == 2:
            self.left()
        else:
            self.right()

    def getMoveTo(self, hijo: 'Tablero') -> int:
        if self.canMoveUp():
            g = Tablero(self.getMatrix())
            g.up()
            if g == hijo:
                return 0
        if self.canMoveDown():
            g = Tablero(self.getMatrix())
            g.down()
            if g == hijo:
                return 1
        if self.canMoveLeft():
            g = Tablero(self.getMatrix())
            g.left()
            if g == hijo:
                return 2
        return 3

    def maximize(self, alfa, beta, p: int) -> Tuple['Tablero', int]:
        if self.isTerminal("max") or not p:
            return(None, self.utility())
        (estadoMaximo, puntiacionMaxima) = (None, -999)

        p -= 1
        lista = self.getChildren("max")
        for hijo in self.getChildren("max"):
            (_, puntuacion) = hijo.minimize(alfa, beta, p)
            if puntuacion > puntiacionMaxima:
                (estadoMaximo, puntiacionMaxima) = (hijo, puntuacion)
            if puntiacionMaxima >= beta:
                break
            if puntiacionMaxima > alfa:
                alfa = puntiacionMaxima

        return (estadoMaximo, puntiacionMaxima)

    def minimize(self, alfa, beta, p: int) -> Tuple['Tablero', int]:
        if self.isTerminal("min") or not p:
            return(None, self.utility())
        (estadoMinimo, puntuacionMinima) = (None, math.inf)

        p -= 1
        for hijo in self.getChildren("min"):
            (_, puntuacion) = hijo.maximize(alfa, beta, p)
            if puntuacion < puntuacionMinima:
                (estadoMinimo, puntuacionMinima) = (hijo, puntuacion)
            if puntuacionMinima <= alfa:
                break
            if puntuacionMinima < beta:
                beta = puntuacionMinima

        return (estadoMinimo, puntuacionMinima)

    def getBestMove(estado: 'Tablero', profundidad: int = 3) -> int:
        (hijo, _) = estado.maximize(-999, math.inf, profundidad)
        return estado.getMoveTo(hijo)

    # def getBestPlace(estado: 'Tablero', profundidad: int = 6) -> int:
    #     (hijo, _) = estado.minimize(-999, math.inf, profundidad)
    #     return estado.setMatrix(hijo.getMatrix())

    def isWin(self) -> bool:
        return 2048 in self.matrix

    def getFreeTiles(self) -> List[Tuple]:
        lista = []
        for i in range(4):
            for j in range(4):
                if not self.matrix[i][j]:
                    lista.append((i, j))
        return lista


    def placeTileRandom(self) -> None:
        movs = self.getFreeTiles()
        num = 0
        randNum = rd.randint(0,9)
        if not randNum:
            num = 4
        else:
            num = 2
        rand = rd.randint(0, len(movs) - 1)
        self.placeTile(movs[rand][0], movs[rand][1], num)


In [29]:
from IPython.display import HTML, display, clear_output
from IPython.html.widgets.interaction import interact

In [30]:
class Visor:
    def __init__(self, tablero: Tablero):
        self.tablero = tablero
    
    def initHtml(self):
        self.html = '<svg height="400" width="400">'

    def draw(self):
        pos = {
            0: "12.5",
            1: "37.5",
            2: "62.5",
            3: "87.5"
        }

        colores = {
            0: "silver",
            2: "ivory",
            4: "antiquewhite",
            8: "lightsalmon",
            16: "darksalmon",
            32: "tomato",
            64: "orangered",
            128: "wheat",
            256: "yellow",
            512: "gold",
            1024: "goldenrod",
            2048: "orange"
        }

        self.initHtml()
        
        for i in range(4):
            for j in range(4):
                self.html += '<rect x="{:d}" y="{:d}" width="{:d}" height="{:d}" stroke="white" stroke-width="5px" fill="{:s}"/>'.format(j * 100, i * 100, 100, 100, colores[self.tablero.matrix[i][j]])
                if self.tablero.matrix[i][j]:
                    self.html += '<text x="{:s}%" y="{:s}%" dominant-baseline="middle" text-anchor="middle" font-size="24">{:d}</text>'.format(pos[j], pos[i], self.tablero.matrix[i][j])
        self.html += '</svg>'
        clear_output(wait=True)
        display(HTML(self.html))

In [31]:
# matriz = [
#     [2,0,0,0],
#     [0,0,0,0],
#     [0,2048,0,0],
#     [0,0,0,0]
# ]
# tab = Tablero(matriz)

# v = Visor(tab)

# v.draw()

In [32]:
def initTablero() -> Tablero:
    matriz = [
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0]
    ]
    tab = Tablero(matriz)
    tab.placeTileRandom()
    tab.placeTileRandom()
    return tab

def main():
    print("Introduce una M para jugar en modo manual, o una A para el modo automático: ")
    
    opcion = ''
    tab = initTablero()
    v = Visor(tab)
    movs = 0
    movimientos = {
        0: "Arriba",
        1: "Abajo",
        2: "Izquierda",
        3: "Derecha"
    }

    movimientosManual = {
        'w': 0,
        's': 1,
        'a': 2,
        'd': 3
    }
    
    while(opcion != 'M' and opcion != 'm' and 
          opcion != 'A' and opcion != 'a'):
          opcion = input()

    # print(np.matrix(tab.matrix))
    v.draw()
    if opcion == 'a' or opcion == 'A':
        while True:
            if tab.isGameOver():
                # print("Puntuacion: {:d}".format(tab.punt))
                # print("Numero de movimientos: {:d}".format(movs))
                # print(np.matrix(tab.matrix))
                v.draw()
                # print("Perdiste!")
                break
            else:
                mov = tab.getBestMove()
                tab.move(mov)
                movs += 1
                # print("\nMovimiento: " + movimientos[mov])
            if tab.isWin():
                # print("Puntuacion: {:d}".format(tab.punt))
                # print("Numero de movimientos: {:d}".format(movs))
                # print(np.matrix(tab.matrix))
                v.draw()
                # print("¡Victoria!")
                break
            else:
                tab.placeTileRandom()
            # print("Puntuacion: {:d}".format(tab.punt))
            # print("Numero de movimientos: {:d}".format(movs))
            # print(np.matrix(tab.matrix))
            v.draw()
    else:
        while True:
            if tab.isGameOver():
                # print("Puntuacion: {:d}".format(tab.punt))
                # print("Numero de movimientos: {:d}".format(movs))
                # print(np.matrix(tab.matrix))
                v.draw()
                break
            else:
                mov = input()
                while mov not in movimientosManual.keys():
                    mov = input()
                mov = movimientosManual[mov]
                tab.move(mov)
                movs += 1
                # print("\nMovimiento: " + movimientos[mov])
            if tab.isWin():
                # print("Puntuacion: {:d}".format(tab.punt))
                # print("Numero de movimientos: {:d}".format(movs))
                # print(np.matrix(tab.matrix))
                v.draw()
                # print("¡Victoria!")
                break
            else:
                tab.placeTileRandom()
            # print("Puntuacion: {:d}".format(tab.punt))
            # print("Numero de movimientos: {:d}".format(movs))
            # print(np.matrix(tab.matrix))
            v.draw()


if __name__ == "__main__":
    main()

KeyboardInterrupt: Interrupted by user